### Text Mining Homework 5

Regarding first 2,000 overviews of movie review data (movies_metadata.csv), after undergoing the following preprocessing, you want to do cluster analysis for movie titles. Create a TF-IDF vector using the method below and answer the questions below

```
[Preprocessing]
[Step 1] Null Remove row with value
[Step 2] Convert non-alphabetic characters to spaces
[Step 3] Convert to lower case
[Step 4] Create TF-IDF considering stopwords
```

#### Problem 1

Please answer the questions below regarding the movie [Jumanji]

**1.1**  Find how many movies are in the same cluster as the movie jumanji using Euclidean similarity and K-means (k=10) method.

**1.2** Find how many movies are in the same cluster as the movie Jumanji using Euclidean similarity and hierarchical clustering (ward, k=10) method.

**1.3** Recommend 3 movies that are closest to the movie Jumanji using cosine similarity 

#### Problem 2

We intend to analyze using a topic model based on LDA . Here, the max features of the TF -IDF vector are limited to 1000 and applied. Answer the questions below

**2.1** Apply the topic model of the Sklearn library to find where the first document falls under topics 0 to 9. Apply (designate 10 topics , random state = 777, running method = online, max iter = 5)

**2.2** Print the top 5 words for each of the 10 topics

**2.3** Use the show_perflexity function of the lecture notes to create a graph with the number of topic candidates from 2 to 30